In [23]:
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')

In [3]:
dat = {'month': ['January', 'February', 'March', 
         'April','May','June',
         'July', 'August','September',
         'October', 'November', 'December'],
       'nrDays': [31, 28, 31,
          30, 31, 30,
          31, 31, 30,
          31, 30, 31]
      }
monthDays = pd.DataFrame(dat)

days = ['01','02','03','04','05','06','07','08','09','10',
        '11','12','13','14','15','16','17','18','19','20',
        '21','22','23','24','25','26','27','28','29','30', '31']

In [26]:
%%time

# Get list of files to use
fileList = glob.glob("Results*")
dataNewFormat = pd.DataFrame()

for i in fileList:
    #print(i)
    xls = pd.ExcelFile(i)
    month = i.split()[5]
    for ii in range(1,monthDays.loc[monthDays['month'] == month].iloc[0][1]+1):
        #print(ii)
        df = xls.parse(days[ii-1] + ' ' + month)
        dataNewFormat = pd.concat([dataNewFormat, df])

CPU times: user 14.1 s, sys: 206 ms, total: 14.4 s
Wall time: 14.4 s


In [27]:
dataNewFormat['Date'] = pd.to_datetime(dataNewFormat['Date'], format="%d.%m.%Y", errors='coerce')
dataNewFormat['Date'] = pd.to_datetime(dataNewFormat['Date']).dt.date

In [28]:
dataNewFormat = dataNewFormat.loc[dataNewFormat['Auction ID'] != 'Auction ID']
dataNewFormat = dataNewFormat.loc[dataNewFormat['Time'] != '02:00 - 02:00']
dataNewFormat = dataNewFormat.loc[dataNewFormat['Time'] != '01:00 - 03:00']
dataNewFormat = dataNewFormat.fillna(0)
dataNewFormat['Connection'] = dataNewFormat['Auction ID'].str[:4]
dataNewFormat['Hour'] = dataNewFormat['Time'].str[:2]

In [32]:
%%time

fileList = glob.glob("DiffResults*")
dataOldFormat = pd.DataFrame()

for i in fileList:
    #print(i)
    xls = pd.ExcelFile(i)
    month = i.split()[5]
    for ii in range(1,monthDays.loc[monthDays['month'] == month].iloc[0][1]+1):
        #print(ii)
        df = xls.parse(days[ii-1] + ' ' + month, header=None)
        mask = df[0].str.contains('Auction', case=False, na=False)
        df[9] = df[mask][0]
        df[9] = df[9].fillna(method='ffill')
        df = df[df[1].notnull()]
        df = df.loc[df[1] != 'Time']
        dataOldFormat = pd.concat([dataOldFormat, df])

CPU times: user 8.49 s, sys: 94.5 ms, total: 8.59 s
Wall time: 8.59 s


In [33]:
dataOldFormat[9] = dataOldFormat[9].str.replace('Auction ID ','')

In [34]:
dataOldFormat[10] = '0'
dataOldFormat1 = dataOldFormat[[9,0,1,2,3,4,5,10,6,7,8]]
dataOldFormat1['Connection'] = dataOldFormat1[9].str.split('-').str[0]
dataOldFormat1['Hour'] = dataOldFormat1[1].str[:2]

In [35]:
dataOldFormat1.columns = ['Auction ID', 'Date', 'Time', 'Offered Capacity [MW]',
                            'Total Requested Capacity [MW]', 'Total Allocated Capacity [MW]',
                            'Auction Clearing Price [EUR/MWh]', 'Congestion Income [EUR]',
                            'Number of Auction Participants', 'Number of Successful Participants',
                            'Number of Auction Bids', 'Connection', 'Hour']

In [36]:
dataOldFormat1['Date'] = pd.to_datetime(dataOldFormat1['Date']).dt.date
dataOldFormat1['Connection'] = dataOldFormat1['Connection'].str.strip()

In [37]:
dataCombined = pd.concat([dataNewFormat, dataOldFormat1])

In [38]:
d = {'Connection': dataCombined['Connection'].unique(), 
     'Export': ['BA','HR','ME','BA','AL','ME','XK','ME',
                'XK','AL','AL','GR','GR','TR','MK','GR',
                'IT','ME','MK','XK'],
     'ExportFull': ['Bosnia','Croatia','Montenegro','Bosnia','Albania','Montenegro','Kosovo','Montenegro',
                    'Kosovo','Albania','Albania','Greece','Greece','Turkey','Macedonia','Greece',
                    'Italy','Montenegro','Macedonia','Kosovo'],
     'Import': ['HR','BA','BA','ME','ME','AL','ME','XK',
                'AL','XK','GR','AL','TR','GR','GR','MK',
                'ME','IT','XK','MK'],
     'ImportFull': ['Croatia','Bosnia','Bosnia','Montenegro','Montenegro','Albania','Montenegro','Kosovo',
                    'Albania','Kosovo','Greece','Albania','Turkey','Greece','Greece','Macedonia',
                    'Montenegro','Italy','Kosovo','Macedonia']
    }
connects = pd.DataFrame(data=d)

In [39]:
dataCombined1 = dataCombined.merge(connects, left_on='Connection', right_on='Connection')

In [44]:
dataCombined1.columns = ['AuctionID', 'Date', 'Time', 'Offered[MW]',
       'Requested[MW]', 'Allocated[MW]',
       'Price[EUR/MWh]', 'CongestionIncome[EUR]',
       'NrParticipants', 'NrSuccessParticipants',
       'NrBids', 'Connection', 'Hour', 'Export', 'ExportFull',
       'Import', 'ImportFull']

In [41]:
#dataCombined5.head(-1)

In [45]:
dataCombined1.to_csv("totalData-v1.csv")